# Margin-implied price level approximation

## Price level approximation

Assumptions
  - Assume risk factors stay constant (which is the case for the log-normal model anyway).
  - Just focus on open position for now (ignore orders - these get cancelled upon closeout anyway and party's margin gets revaluated again to see if margin > maintenance in which case position doesn't get closed out).

Need to work out price `S` so that:

$$ \text{margin_level}(S)=\text{account_balance} + \text{P&L}(S) $$

where for futures:

$$ \text{margin_maintenance}(S)= max(|\text{open_volume}| * \text{slippage_per_unit}, 0) + |\text{open_volume}| * \text{risk_factor} * S $$
and

$$ \text{P&L}(S)= \text{open_volume}*[S-S_{\text{current}}] $$

### First approximation

Assumptions:
   - Assume margin levels don't get affected by price moves

As per assumptions above just calculate the P&L loss that'd bring margin account balance below **current** maintenance margin so:


### Second approximation

Assumptions:
  - Assume slippage stays constant from when the position opens.

Let:

$$ c_1 = max(|\text{open_volume}| * \text{slippage_per_unit}, 0) $$
$$ c_2 = |\text{open_volume}| * \text{risk_factor} $$

and $c_{level}$ be the scaling for a particular risk level, hence we get:

$$S = \frac{\text{account_balance}-\text{open_volume}*S_\text{current}-c_{level}*c_1}{c_{level}*c_2-\text{open_volume}}$$

### Third approximation

Same as above, but use live slippage (NOTE: this is just to see how volatile this metric is in a typical market, in reallity some max slippage observed over a sliding time window would likely be better as in case of a sudden market moves spreads are likely to widen thus increasing the slippage). 

## Implementation

### First approximation

AttributeError: 'MarketMock' object has no attribute 'risk_factor_sell'

## Appendix: Maintenance margin calculation

```maintenance_margin = max ( maintenance_margin_long, maintenance_margin_short)```

riskiest long = max( open_volume + buy_orders , 0 )

riskiest short = min( open_volume + sell_orders, 0 )

### `maintenance_margin_long`
(calculation for `maintenance_margin_short` is equivalent, see [spec](https://github.com/vegaprotocol/specs-internal/blob/master/protocol/0019-margin-calculator.md) for details)

```maintenance_margin_long_open_position = max(slippage_volume * slippage_per_unit, 0) + slippage_volume * [ quantitative_model.risk_factors_long ] . [ Product.value(market_observable) ]```,

```maintenance_margin_long_open_orders = buy_orders * [ quantitative_model.risk_factors_long ] . [ Product.value(market_observable) ]  ```,

where

```slippage_volume =  max( open_volume, 0 ) ```,

and

if ```open_volume > 0```  then 

```slippage_per_unit =  Product.value(market_observable) - Product.value(exit_price) ```, 

else ```slippage_per_unit = 0```.

In [ ]:
import sys; sys.path.append('../')
from sim.mechanism.margin_calculations import *

market = MarketMock(mark_price=100, slippage=-10, factor_long=0.1, factor_short=0.2, margin_scalings=MarginScalings(1.1,1.5,2.0))
initial_deposit=1000
party1 = Party("party1", market, initial_deposit)
position=-10
if not party1.try_open_position(position):
    print("Insufficient funds to open a position")
    exit()

print(party1.margin_levels())
print(party1.accounts())
print(party1.price_levels())

new_mark_price=134
market.set_mark_price(167)
print("Mark price updated to ", new_mark_price, '\n')
party1.update()
print(party1.accounts())
print(party1.margin_levels())
print(party1.price_levels())